In [234]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
from sklearn.datasets import load_iris
import torch
from sklearn.preprocessing import LabelEncoder, StandardScaler
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split

In [235]:
df = load_iris()

In [236]:
iris_df = pd.DataFrame(data=df.data, columns = df.feature_names)
iris_df['iris_target'] = df.target

In [237]:
iris_df.describe()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),iris_target
count,150.000000,150.000000,150.000000,150.000000,150.000000
mean,5.843333,3.057333,3.758000,1.199333,1.000000
std,0.828066,0.435866,1.765298,0.762238,0.819232
min,4.300000,2.000000,1.000000,0.100000,0.000000
25%,5.100000,2.800000,1.600000,0.300000,0.000000
50%,5.800000,3.000000,4.350000,1.300000,1.000000
75%,6.400000,3.300000,5.100000,1.800000,2.000000
max,7.900000,4.400000,6.900000,2.500000,2.000000


In [238]:
iris_df.isnull().sum()

sepal length (cm)    0
sepal width (cm)     0
petal length (cm)    0
petal width (cm)     0
iris_target          0
dtype: int64

In [239]:
iris_df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),iris_target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [240]:
x = iris_df[['sepal length (cm)','sepal width (cm)','petal length (cm)','petal width (cm)']]

In [241]:
y = iris_df[['iris_target']]

In [242]:
x = x.values
y = y.values

In [243]:
labelencoder = LabelEncoder()
y = labelencoder.fit_transform(y)

/home/user/miniconda/lib/python3.9/site-packages/sklearn/preprocessing/_label.py:110: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [308]:
y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [244]:
x.shape

(150, 4)

In [245]:
scaler = StandardScaler()
x = scaler.fit_transform(x)

In [246]:
X = torch.tensor(x,dtype=torch.float32)
Y = torch.tensor(y,dtype=torch.long)

In [247]:
X = X.unsqueeze(1)


In [248]:
X.shape

torch.Size([150, 1, 4])

In [249]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size = 0.2, random_state=123)


In [250]:
train_loader = DataLoader(TensorDataset(X_train, Y_train), batch_size = 10, shuffle = True)
test_loader = DataLoader(TensorDataset(X_test, Y_test),batch_size = 10, shuffle = False)

In [251]:
# Define the RNN model
class RNN(nn.Module):
    def __init__(self):
        super(RNN, self).__init__()
        self.rnn = nn.RNN(input_size=4, hidden_size=10, batch_first=True)
        self.fc = nn.Linear(10, 3)

    def forward(self, x):
        _, h_n = self.rnn(x)
        out = self.fc(h_n.squeeze(0))
        return out

In [252]:
model = RNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [253]:
# Train the model
num_epochs = 150
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")

In [297]:
X_batch

tensor([[[-0.7795,  0.7888, -1.3402, -1.3154]],

        [[ 0.5533, -0.5924,  0.7628,  0.3958]],

        [[-1.2642,  0.7888, -1.0560, -1.3154]],

        [[ 0.3110, -0.1320,  0.6491,  0.7907]],

        [[-0.9007,  1.7096, -1.2266, -1.3154]],

        [[-0.5372,  0.7888, -1.1697, -1.3154]],

        [[-0.5372,  0.7888, -1.2834, -1.0522]],

        [[-0.2948, -0.5924,  0.6491,  1.0539]],

        [[ 2.2497,  1.7096,  1.6722,  1.3172]],

        [[-1.0218,  1.2492, -1.3402, -1.3154]]])

In [254]:
# Evaluate the model
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for X_batch, y_batch in test_loader:
        outputs = model(X_batch)
        _, predicted = torch.max(outputs, 1)
        total += y_batch.size(0)
        correct += (predicted == y_batch).sum().item()

print(f"Test Accuracy: {100 * correct / total:.2f}%")

In [255]:
new_df = iris_df.groupby(iris_df['iris_target'])

In [258]:
new_df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),iris_target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0
50,7.0,3.2,4.7,1.4,1
51,6.4,3.2,4.5,1.5,1
52,6.9,3.1,4.9,1.5,1
53,5.5,2.3,4.0,1.3,1
54,6.5,2.8,4.6,1.5,1


In [261]:
group1 = iris_df[iris_df['iris_target'] == 0]
group2 = iris_df[iris_df['iris_target'] == 1]
group3 = iris_df[iris_df['iris_target'] == 2]
group4 = iris_df[iris_df['iris_target'] == 3]

In [264]:
group1.shape

(50, 5)

In [287]:
group2.describe()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),iris_target
count,50.000000,50.000000,50.000000,50.000000,50.0
mean,5.936000,2.770000,4.260000,1.326000,1.0
std,0.516171,0.313798,0.469911,0.197753,0.0
min,4.900000,2.000000,3.000000,1.000000,1.0
25%,5.600000,2.525000,4.000000,1.200000,1.0
50%,5.900000,2.800000,4.350000,1.300000,1.0
75%,6.300000,3.000000,4.600000,1.500000,1.0
max,7.000000,3.400000,5.100000,1.800000,1.0


In [288]:
inp = np.array([5.936,2.77,4.2,1.3]).reshape(1,4)

In [289]:
inp.shape

(1, 4)

In [290]:
inp = scaler.transform(inp)

In [291]:
inp

array([[ 0.11228223, -0.66143204,  0.25122143,  0.13250973]])

In [292]:
inpX = torch.tensor(inp,dtype=torch.float32)

In [293]:
inpX = inpX.unsqueeze(1)

In [294]:
inpX

tensor([[[ 0.1123, -0.6614,  0.2512,  0.1325]]])

In [295]:
model(inpX)

tensor([[ 0.0429, -0.0190, -0.0928]], grad_fn=<AddmmBackward0>)

In [296]:
probabilities = torch.softmax(model(inpX), dim=1)
predicted_class = torch.argmax(model(inpX), dim=1).item()
print(f"Predicted class: {predicted_class}")


Predicted class: 0


In [300]:
m = np.array([[-0.7795,  0.7888, -1.3402, -1.3154],[0.5533, -0.5924,  0.7628,  0.3958],[-1.2642,  0.7888, -1.0560, -1.3154]]).reshape(3,4)

m

In [301]:
m

array([[-0.7795,  0.7888, -1.3402, -1.3154],
       [ 0.5533, -0.5924,  0.7628,  0.3958],
       [-1.2642,  0.7888, -1.056 , -1.3154]])

In [302]:
m = torch.tensor(m,dtype=torch.float32)

In [303]:
m = m.unsqueeze(1)

In [304]:
m

tensor([[[-0.7795,  0.7888, -1.3402, -1.3154]],

        [[ 0.5533, -0.5924,  0.7628,  0.3958]],

        [[-1.2642,  0.7888, -1.0560, -1.3154]]])

In [305]:
model(m)

tensor([[ 0.1946,  0.0847, -0.2697],
        [-0.0429,  0.0074, -0.0842],
        [ 0.2191,  0.0676, -0.2850]], grad_fn=<AddmmBackward0>)

In [309]:
_, predicted = torch.max(model(inpX), 1)

In [310]:
predicted

tensor([0])

In [311]:
# Save the model
torch.save(model.state_dict(), 'rnn_iris_model.pth')
print("Model saved as rnn_iris_model.pth")

Model saved as rnn_iris_model.pth
